# **Text Classification Model Deployment using FastAPI and Gradio**

**HuggingFace Repo and App Link: https://huggingface.co/spaces/HussainM899/AG_News_Text_Classifier_FastAPI**

In [ ]:
!pip install joblib pickles gradio sklearn fastapi

  Using cached pickles-0.1.1-py3-none-any.whl (5.3 kB)
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- ### Importing Libraries

In [ ]:
# Basic imports for data manipulation and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# scikit-learn imports for model loading and possibly preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Joblib or Pickle for loading your trained model
import joblib
import pickle

In [ ]:
import os
print("Current Working Directory: ", os.getcwd())
# List files in the current directory
print("Files in Current Directory: ", os.listdir('.'))


Current Working Directory:  /content/AG_News_Text_Classifier_FastAPI
Files in Current Directory:  ['.ipynb_checkpoints', '.git', '.gitattributes']


In [ ]:
!pip install joblib gradio fastapi nltk

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

- ### **Pre-Processing and Classification Modelling**

In [ ]:
# Import the necessary libraries for preprocessing and deployment
import re
import joblib
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from fastapi import FastAPI

# Define the custom function for text cleaning
def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove non-alphabetic characters and lowercase the text
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A).lower()
    # Tokenization
    tokens = text.split()
    # Remove stopwords and lemmatize
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Load your trained model
model = joblib.load('/content/drive/MyDrive/Data Science/MLOPS/Models/text_classification_LR_model (1).joblib')

# Load the TF-IDF vectorizer
tfidf_vectorizer = joblib.load('/content/drive/MyDrive/Data Science/MLOPS/Models/tfidf_vectorizer.joblib')

# Preprocessing function for input text
def preprocess(input_text):
    # Apply text cleaning
    input_text = clean_text(input_text)
    input_text = [input_text]
    # Transform input text using TF-IDF vectorizer
    input_text = tfidf_vectorizer.transform(input_text)
    return input_text

# Predict the class for the input text
def predict_class(input_text):
    input_text = preprocess(input_text)
    prediction = model.predict(input_text)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    predicted_class = classes[prediction[0]]
    return predicted_class

# FastAPI app
app = FastAPI()

@app.get('/')
async def welcome():
    return "Welcome to the Text Classification API"

@app.post('/classify_text')
async def classify_text(input_text: str):
    prediction = predict_class(input_text)
    return {"classification": prediction}


- ### **Gradio App Deployment**

In [ ]:
!pip install gradio deploy

In [ ]:
import gradio as gr

# Create Gradio interface
iface = gr.Interface(fn=predict_class,
                     inputs="text",
                     outputs="text",
                     title="Text Classification API",
                     description="Enter text to classify it into categories: World, Sports, Business, Sci/Tech.")
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6f1c2c3f876b556c98.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
